In [1]:
import pretty_midi
import numpy as np
from pathlib import Path

INPUT_MIDI = Path("../data/input_midi/example_01.mid")
OUTPUT_MIDI = Path("../data/output_midi/example_01_ai.mid")

TEMPO = 120
TIME_SIG = (4, 4)


In [2]:
midi = pretty_midi.PrettyMIDI(str(INPUT_MIDI))
instrument = midi.instruments[0]
notes = instrument.notes

print(f"Notes: {len(notes)}")
print(f"Pitch range: {min(n.pitch for n in notes)}–{max(n.pitch for n in notes)}")
print(f"Duration: {midi.get_end_time():.2f}s")

Notes: 32
Pitch range: 60–69
Duration: 16.00s


In [3]:
pitches = [n.pitch for n in notes]
avg_pitch = np.mean(pitches)
num_notes = len(notes)

seconds_per_bar = 60 / TEMPO * TIME_SIG[0]
bars = int(round(midi.get_end_time() / seconds_per_bar))

def density_bucket(n):
    if n < 20:
        return "low"
    elif n < 60:
        return "medium"
    return "high"

def register_bucket(p):
    if p < 50:
        return "low"
    elif p < 70:
        return "mid"
    return "high"

features = {
    "density": density_bucket(num_notes),
    "register": register_bucket(avg_pitch),
    "bars": bars,
}

features

{'density': 'medium', 'register': 'mid', 'bars': 8}

In [4]:
def decide_action(features):
    if features["density"] == "high":
        return {
            "action": "play",
            "role": "counter_melody",
            "density": "low",
            "register": "lower",
            "bars": features["bars"],
        }
    return {
        "action": "play",
        "role": "counter_melody",
        "density": "medium",
        "register": "same",
        "bars": features["bars"],
    }

action = decide_action(features)
action

{'action': 'play',
 'role': 'counter_melody',
 'density': 'medium',
 'register': 'same',
 'bars': 8}

In [5]:
def generate_response(action, start_time):
    response = pretty_midi.PrettyMIDI()
    inst = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program("Electric Piano 1")
    )

    # simple D minor tones for MVP
    scale = [62, 65, 69]
    bar_len = 60 / TEMPO * TIME_SIG[0]

    t = start_time
    notes_per_bar = 1 if action["density"] == "low" else 2

    for _ in range(action["bars"]):
        for i in range(notes_per_bar):
            pitch = np.random.choice(scale)
            note = pretty_midi.Note(
                velocity=70 - i * 10,
                pitch=pitch,
                start=t + i * 0.5,
                end=t + i * 0.5 + 0.4,
            )
            inst.notes.append(note)
        t += bar_len

    response.instruments.append(inst)
    return response

In [6]:
start_time = midi.get_end_time()
response_midi = generate_response(action, start_time)

combined = pretty_midi.PrettyMIDI()
combined.instruments = midi.instruments + response_midi.instruments

OUTPUT_MIDI.parent.mkdir(parents=True, exist_ok=True)
combined.write(str(OUTPUT_MIDI))

OUTPUT_MIDI

PosixPath('../data/output_midi/example_01_ai.mid')